**将列表数组转化为numpy格式**

List转Numpy：numpy.array(list,dtype = float)

Numpy转List：array.tolist()

**模型训练三大段**


In [ ]:
model=tf.keras.Sequential([keras.layers.Dense(units = 1,input_shape = [1])])

model.compile(optimizer = 'sgd',loss = 'mean_squared_error')
#优化器sgd一般情况下比adam好
model.fit(x_train,y_train,epochs = 1000)



***搭建卷积神经网路模型***

初学CNN 激活函数采用relu

CNN模型的输入shape为3维 eg.(28,28,1) 尺寸28*28 通道个数为1（灰度）、3（彩色）
而且模型中输入的值最好在0-1之间   有利于模型拟合
模型第一层网络内要注明input_data的形状维度
cnn搭建好后 需要注入一层flatten  2维拉直为1维
最后一层网络Dense个数为classification的个数 激活函数通常采用softmax，但若属于二分类问题 则最后一层神经元只有一个

model.compile中优化器用adam、sgd   adam、RSMprop的学习率可以自动修正

loss函数用交叉熵 'sparse_categorical_crossentropy'
衡量标准 一般选metrics = ['accuracy']

model.fit喂入数据进行训练  设置batch_size epoch  callback
callback函数可以重定义  比如训练到准确率大于多少时停止 或者从上次未完成的训练中断点续训
model.evaluate 喂入测试集进行验证

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train , y_train) , (x_test , y_test) = mnist.load_data ()
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.99):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True
print(x_train.shape)
print(x_test.shape)
x_train=x_train.reshape(60000,28,28,1)
x_test=x_test.reshape(10000,28,28,1)
x_train=x_train/255.0
x_test=x_test/255.0

model = tf.keras.models.Sequential ([
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',input_shape = (28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D (2 , 2) ,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation = 'relu'),
    tf.keras.layers.Dense(10,activation = 'softmax')
])

model.compile (optimizer = 'adam' ,
               loss = 'sparse_categorical_crossentropy' ,
               metrics = ['accuracy'])
model.summary()
model.fit(x_train,y_train,batch_size = 128,epochs = 1000,callbacks = [myCallback()],validation_data = (x_test,y_test),validation_freq =2)
test_loss = model.evaluate(x_test, y_test)##



***python读取数据集的解压操作***

eg. 以人马识别分类的数据集为例

In [ ]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

***数据预处理***
采用ImageDataGenerator生成器

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)
#rescale=1/255 是为了将数据归一化到0-1之间
# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')
#人马 分为两类  class_mode为二分类 binary
#'/tmp/horse-or-human/'目录下有两个文件夹 horses和humans 恰好为两类
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

采用ImageDataGenerator时  需要修改model.fit


In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)


*****可视化*****

*对输入图像的可视化*



In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

pic_index = 0 # Index for iterating over images
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )
pic_index+=8

next_cat_pix = [os.path.join(train_cats_dir, fname)
                for fname in train_cat_fnames[ pic_index-8:pic_index]
               ]

next_dog_pix = [os.path.join(train_dogs_dir, fname)
                for fname in train_dog_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

*对训练效果的数据曲线可视化*

训练结果保存在history中，
通常history=model.fit()

方法一：

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'acc' ]
val_acc  = history.history[ 'val_acc' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

方法二

In [ ]:
# 显示训练集和验证集的acc和loss曲线
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

*对训练结果进行保存*
下次训练时 直接在之前的基础上继续训练

In [ ]:
checkpoint_save_path = "./checkpoint/fashion.ckpt"
if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True)
#调用方式
model.fit(callbacks = [cp_callback])
#callback参数属于一个列表，可以添加多个回调函数，实现多功能
#eg.
model.fit(callbacks=[cp_callback,myCallback()])

回调函数可以自定义重载
eg. 设置当accuracy或者loss达到一定值时 停止训练 训练结束

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.99):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True

*注：* logs.get('accuracy')中 'accuracy'取决于model.compile的metric值的设定

二分类时 metric='accuracy' 多分类时 常用metric='sparse_categorical_accuracy'
